In [120]:
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2
import os
import re


In [121]:
mp_pose = mp.solutions.pose


In [122]:
landmarks_points = {
    "nose": 0,
    "left_shoulder": 11, "right_shoulder": 12,
    "left_elbow": 13, "right_elbow": 14,
    "left_wrist": 15, "right_wrist": 16,
    "left_hip": 23, "right_hip": 24,
    "left_knee": 25, "right_knee": 26,
    "left_ankle": 27, "right_ankle": 28,
    "left_heel": 29, "right_heel": 30,
    "left_foot_index": 31, "right_foot_index": 32,
}

col_names = []
for i in range(len(landmarks_points.keys())):
    name = list(landmarks_points.keys())[i]
    col_names.append(name + "_x")
    col_names.append(name + "_y")
    col_names.append(name + "_z")
    col_names.append(name + "_v")
col_names.append("pose") # from 0 to n
col_names.append("correct") # correct: 0, wrong: 1
col_names.append("step") # from 0 to n


In [123]:
full_list = []

main_dir = "./poses_dataset/Images"
pose_dir_list = os.listdir(main_dir)

for i in range(0, len(pose_dir_list)):
    wrong_right_dir_list = os.listdir(f"{main_dir}/{pose_dir_list[i]}")

    for j in range(0, len(wrong_right_dir_list)):
        steps_dir_list = os.listdir(
            f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}")

        for k in range(0, len(steps_dir_list)):
            images_dir_list = os.listdir(
                f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}")
            if j == 1:
                reg = re.search(r"([0-9])", steps_dir_list[k])

            for l in range(0, len(images_dir_list)):
                pre_list = []
                with mp_pose.Pose(static_image_mode=True, enable_segmentation=True) as pose:
                    image = cv2.imread(
                        f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}/{images_dir_list[l]}")
                    result = pose.process(
                        cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    try:
                        predict = True
                        for landmarks in result.pose_landmarks.landmark:
                            pre_list.append(landmarks)
                    except AttributeError:
                        print(
                            f"No points {main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}/{images_dir_list[l]}")
                        predict = False

                if predict == True:
                    gen1116 = np.array([
                        [
                            pre_list[m].x,
                            pre_list[m].y,
                            pre_list[m].z,
                            pre_list[m].visibility
                        ] for m in range(11, 17)
                    ]).flatten().tolist()

                    gen2333 = np.array([
                        [
                            pre_list[m].x,
                            pre_list[m].y,
                            pre_list[m].z,
                            pre_list[m].visibility
                        ] for m in range(23, 33)
                    ]).flatten().tolist()

                    gen1116.extend(gen2333)

                    all_list = [
                        pre_list[0].x,
                        pre_list[0].y,
                        pre_list[0].z,
                        pre_list[0].visibility,
                    ]

                    all_list.extend(gen1116)
                    all_list.append(i)
                    if j == 1:
                        all_list.append(reg.group(1))
                    else:
                        all_list.append(j)
                    all_list.append(k)
                    full_list.append(all_list)


No points ./poses_dataset/Images/Anantasana/Anantasana_Right_Steps/Anantasana_Right_Step_1/20230317_072737(0).jpg
No points ./poses_dataset/Images/Anantasana/Anantasana_Right_Steps/Anantasana_Right_Step_1/20230317_072738.jpg
No points ./poses_dataset/Images/Anantasana/Anantasana_Right_Steps/Anantasana_Right_Step_1/20230317_072853(0).jpg
No points ./poses_dataset/Images/Anantasana/Anantasana_Right_Steps/Anantasana_Right_Step_1/20230317_072900.jpg
No points ./poses_dataset/Images/Anantasana/Anantasana_Right_Steps/Anantasana_Right_Step_1/20230317_072903.jpg
No points ./poses_dataset/Images/Anantasana/Anantasana_Right_Steps/Anantasana_Right_Step_1/20230317_073000.jpg
No points ./poses_dataset/Images/Anantasana/Anantasana_Right_Steps/Anantasana_Right_Step_2/20230317_075254.jpg
No points ./poses_dataset/Images/Anantasana/Anantasana_Right_Steps/Anantasana_Right_Step_2/20230317_075328(0).jpg
No points ./poses_dataset/Images/Anantasana/Anantasana_Right_Steps/Anantasana_Right_Step_3/20230317_075

In [124]:
data = pd.DataFrame(full_list, columns=col_names)
data.to_csv("data.csv", index=False)
